In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser, JsonOutputParser, XMLOutputParser, BaseOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough, ConfigurableField, chain
from langchain_core.runnables.history import RunnableWithMessageHistory, ConfigurableFieldSpec
from langchain_core.exceptions import OutputParserException
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, SystemMessage
from pydantic import BaseModel, Field, model_validator
from typing import List, Iterator
from pprint import pprint
import json
from operator import itemgetter

# base_url = "https://api.deepseek.com/"

# model="deepseek-chat"
# llm = ChatOpenAI(model=model, temperature=0, api_key=api_key, base_url=base_url)

base_url="https://ark.cn-beijing.volces.com/api/v3"

model="doubao-1.5-pro-32k-250115"
llm = ChatOpenAI(model=model, temperature=0, api_key=api_key, base_url=base_url)

# base_url="http://localhost:11434"
# model="deepseek-r1"
# llm = ChatOllama(model=model, temperature=0, base_url=base_url)

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """内存中实现的聊天消息历史记录。"""
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """添加一组消息到存储中"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

In [20]:
store = {}
def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

history = get_by_session_id("1")
history.add_messages([AIMessage(content="你好")])
print(store)
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个擅长{ability}的助手"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
    # ("human", "{question}"),
])
chain = prompt | llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)
print(chain_with_history.invoke(
    {"ability": "math", "question": "余弦函数是什么意思？"},
    config={"configurable": {"session_id": "foo"}}
))
print(store)
print(chain_with_history.invoke(
    {"ability": "math", "question": "他的反函数是什么？"},
    config={"configurable": {"session_id": "foo"}}
))
print(store)

{'1': InMemoryHistory(messages=[AIMessage(content='你好', additional_kwargs={}, response_metadata={})])}
content='余弦函数是数学中一类重要的三角函数，下面从定义、图像、性质等方面为你详细介绍：\n\n### 定义\n- **直角三角形定义**：在一个直角三角形中，对于一个锐角 $\\theta$，余弦函数表示该角的邻边与斜边的比值。假设直角三角形的一个锐角为 $\\theta$，它的邻边长度为 $a$，斜边长度为 $c$，那么 $\\theta$ 的余弦值记为 $\\cos\\theta$，且 $\\cos\\theta=\\frac{a}{c}$。\n- **单位圆定义**：在平面直角坐标系中，以原点 $O$ 为圆心，作一个半径为 $1$ 的单位圆。设角 $\\alpha$ 的顶点与原点重合，始边与 $x$ 轴的正半轴重合，角 $\\alpha$ 的终边与单位圆交于点 $P(x,y)$，那么角 $\\alpha$ 的余弦函数定义为 $\\cos\\alpha = x$。\n- **任意角定义**：设角 $\\alpha$ 是一个任意角，它的终边上任意一点 $P$（不与原点重合）的坐标为 $(x,y)$，点 $P$ 到原点的距离为 $r=\\sqrt{x^{2}+y^{2}}>0$，则角 $\\alpha$ 的余弦函数定义为 $\\cos\\alpha=\\frac{x}{r}$。\n\n### 函数表达式及参数含义\n余弦函数的一般形式为 $y = A\\cos(\\omega x+\\varphi)+k$ ，其中：\n- $A$ 表示振幅，它决定了函数值的波动范围，$|A|$ 越大，函数图像的波动幅度越大；$|A|$ 越小，波动幅度越小。\n- $\\omega$ 影响函数的周期，其周期 $T = \\frac{2\\pi}{|\\omega|}$，$\\omega$ 越大，周期越小，函数变化越快；$\\omega$ 越小，周期越大，函数变化越慢。\n- $\\varphi$ 称为初相，它决定了函数图像的左右平移，当 $\\varphi>0$ 时，图像向左平移 $|\\varphi|$ 个单位；当 $\\varphi<0$ 时，图像向右平移 $|\\va

In [19]:
store = {}
def get_session_history(user_id: str, session_id: str) -> BaseChatMessageHistory:
    if (user_id, session_id) not in store:
        store[(user_id, session_id)] = InMemoryHistory()
    return store[(user_id, session_id)]


prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个擅长{ability}的助手"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
    # ("human", "{question}"),
])
chain = prompt | llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config = [
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="session id",
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="session id",
        ),
    ],
)
print(chain_with_history.invoke(
    {"ability": "math", "question": "余弦函数是什么意思？"},
    config={"configurable": {"user_id": "123", "session_id": "foo"}}
))
print(store)
print(chain_with_history.invoke(
    {"ability": "math", "question": "他的反函数是什么？"},
    config={"configurable": {"session_id": "foo"}}
))
print(store)

content='余弦函数是数学中一类重要的三角函数，下面从定义、图像、性质等方面为你详细介绍：\n\n### 定义\n- **直角三角形定义**：在一个直角三角形中，对于任意一个锐角 $\\theta$，余弦函数表示为 $\\cos\\theta$。它的定义是邻边与斜边的比值。假设直角三角形中，$\\theta$ 所对的直角边为 $a$，另一条直角边（邻边）为 $b$，斜边为 $c$，那么 $\\cos\\theta=\\frac{b}{c}$。\n- **单位圆定义**：在平面直角坐标系中，以原点为圆心作一个半径为 $1$ 的圆，称为单位圆。设角 $\\alpha$ 的顶点与原点重合，始边与 $x$ 轴的正半轴重合，角 $\\alpha$ 的终边与单位圆交于点 $P(x,y)$，那么角 $\\alpha$ 的余弦定义为 $\\cos\\alpha = x$。\n\n### 函数表达式与定义域、值域\n- 余弦函数的一般形式为 $y = A\\cos(\\omega x+\\varphi)+k$（$A\\neq0$，$\\omega\\gt0$），当 $A = 1$，$\\omega = 1$，$\\varphi = 0$，$k = 0$ 时，为最简单的形式 $y=\\cos x$。\n- 定义域：对于函数 $y = \\cos x$，$x$ 可以取任意实数，即定义域为 $(-\\infty,+\\infty)$。\n- 值域：由于单位圆上点的横坐标 $x$ 的取值范围是 $[-1,1]$，所以余弦函数的值域是 $[-1,1]$。\n\n### 函数图像\n余弦函数 $y = \\cos x$ 的图像叫做余弦曲线，它是一条波浪线。该曲线关于 $y$ 轴对称，是一个偶函数。可以通过五点法来绘制其在一个周期内的图像，五个关键点分别是 $(0,1)$，$(\\frac{\\pi}{2},0)$，$(\\pi,-1)$，$(\\frac{3\\pi}{2},0)$，$(2\\pi,1)$ 。\n\n### 函数性质\n- **周期性**：余弦函数是周期函数，其最小正周期是 $T = 2\\pi$，即 $\\cos(x + 2k\\pi)=\\cos x$，$k\\in Z$。\n- **奇偶性**：因为 $\\cos(-x)=\\cos x$，所以余弦函数是偶函数，其图像